# COMP34812 Natural Language Understanding Courseworklow key lemming an stemming: Transformer based approach

In [1]:
!pip install transformers datasets pandas nltk numpy scikit_learn --quiet
!pip install -U tensorflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

In [2]:
import pandas as pd
import nltk
import os
import regex as re
import numpy as np
import tensorflow as tf
from nltk.corpus import stopwords
from transformers import TFBertModel, BertTokenizer
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

max_sequence_length = 512

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
special = tokenizer.special_tokens_map
special

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

## Load and Clean Data

In [5]:
from huggingface_hub import snapshot_download

# set HF_TOKEN in your enviroment
snapshot_download(repo_id="aap9002/NLU-Coursework",  repo_type="dataset", allow_patterns=f"*", local_dir='./')

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

dev.csv:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

'/content'

In [6]:
dev_set = pd.read_csv('dev.csv')
dev_set.head()

,premise,hypothesis,label
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0
4,I know that many of you are interested in addr...,The problems must be addressed,1


In [7]:
train_set = pd.read_csv('train.csv')
train_set.head()

,premise,hypothesis,label
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1


In [8]:
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(text):
    text = str(text)

    text = text.lower()

    text = re.sub(r'[^\w\s]', ' ', text)

    text = nltk.word_tokenize(text)

    processed = []
    for word in text:
        if word in stop_words:
            continue

        word = lemmatizer.lemmatize(word)

        word = word.strip()

        if len(word) < 2:
            continue

        processed.append(word)

    return processed

In [9]:
dev_set['premise_cleaned'] = dev_set['premise'].apply(clean_text)
dev_set['hypothesis_cleaned'] = dev_set['hypothesis'].apply(clean_text)

train_set['premise_cleaned'] = train_set['premise'].apply(clean_text)
train_set['hypothesis_cleaned'] = train_set['hypothesis'].apply(clean_text)

In [10]:
dev_set.head()

,premise,hypothesis,label,premise_cleaned,hypothesis_cleaned
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0,"[starting, soft, underbelly, 16, 000, troop, g...","[general, nelson, mile, 30, 000, troop, attack]"
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1,"[class, broken, light, sweat, gasping, air]","[class, grew, tense, time, went]"
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1,"[samson, famous, haircut, would, find, hard, r...","[unknown, exactly, within, town, samson, recei..."
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0,"[man, black, shirt, hold, baby, blue, shirted,...","[darkly, dressed, man, pass, cry, baby, man, l..."
4,I know that many of you are interested in addr...,The problems must be addressed,1,"[know, many, interested, addressing, issue, le...","[problem, must, addressed]"


## Tokenize with Bert Tokenizer

In [11]:
def create_input(premise_tokens, hypothesis_tokens):
    return tokenizer(' '.join(premise_tokens), ' '.join(hypothesis_tokens), padding="max_length", truncation=True, return_tensors='tf')

dev_embeddings = dev_set.apply(lambda x: create_input(x['premise_cleaned'], x['hypothesis_cleaned']), axis=1)
train_embeddings = train_set.apply(lambda x: create_input(x['premise_cleaned'], x['hypothesis_cleaned']), axis=1)

In [12]:
dev_embeddings.head()

,0
0,"[input_ids, token_type_ids, attention_mask]"
1,"[input_ids, token_type_ids, attention_mask]"
2,"[input_ids, token_type_ids, attention_mask]"
3,"[input_ids, token_type_ids, attention_mask]"
4,"[input_ids, token_type_ids, attention_mask]"


## Model Design

In [13]:

# Define input layers
input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name='attention_mask')
token_type_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name='token_type_ids')

# Get BERT outputs
bert_outputs = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
# Use the pooled output for classification
pooled_output = bert_outputs.pooler_output

# Add additional layers on top
x = tf.keras.layers.Dropout(0.3)(pooled_output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# Final classification layer (adjust num_labels accordingly)
logits = tf.keras.layers.Dense(2, activation='softmax')(x)

# Build the model
model = tf.keras.Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=logits)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 token_type_ids (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                              

In [14]:
train_embeddings

,0
0,"[input_ids, token_type_ids, attention_mask]"
1,"[input_ids, token_type_ids, attention_mask]"
2,"[input_ids, token_type_ids, attention_mask]"
3,"[input_ids, token_type_ids, attention_mask]"
4,"[input_ids, token_type_ids, attention_mask]"
...,...
24427,"[input_ids, token_type_ids, attention_mask]"
24428,"[input_ids, token_type_ids, attention_mask]"
24429,"[input_ids, token_type_ids, attention_mask]"
24430,"[input_ids, token_type_ids, attention_mask]"


In [15]:
train_x = [e.data for e in train_embeddings]
dev_x = [e.data for e in dev_embeddings]

train_x = {k: np.vstack([d[k] for d in train_x]) for k in train_x[0]}
dev_x = {k: np.vstack([d[k] for d in dev_x]) for k in dev_x[0]}

train_y = tf.convert_to_tensor(np.reshape(train_set['label'].values, (-1, 1)))
dev_y = tf.convert_to_tensor(np.reshape(dev_set['label'].values, (-1, 1)))

# extract inner lists of y

train_y = tf.squeeze(train_y)
dev_y = tf.squeeze(dev_y)

In [16]:
history = model.fit(train_x, train_y, epochs=8, batch_size=32, validation_data=(dev_x, dev_y))

Epoch 1/8
764/764 [==============================] - 418s 476ms/step - loss: 0.6790 - accuracy: 0.5752 - val_loss: 0.5940 - val_accuracy: 0.6836
Epoch 2/8
764/764 [==============================] - 354s 464ms/step - loss: 0.5499 - accuracy: 0.7261 - val_loss: 0.5148 - val_accuracy: 0.7487
Epoch 3/8
764/764 [==============================] - 355s 464ms/step - loss: 0.3842 - accuracy: 0.8328 - val_loss: 0.5424 - val_accuracy: 0.7436
Epoch 4/8
764/764 [==============================] - 354s 463ms/step - loss: 0.2100 - accuracy: 0.9185 - val_loss: 0.7679 - val_accuracy: 0.7346
Epoch 5/8
764/764 [==============================] - 354s 463ms/step - loss: 0.1114 - accuracy: 0.9592 - val_loss: 0.9552 - val_accuracy: 0.7352
Epoch 6/8
764/764 [==============================] - 354s 464ms/step - loss: 0.0657 - accuracy: 0.9767 - val_loss: 1.1465 - val_accuracy: 0.7346
Epoch 7/8
764/764 [==============================] - 354s 464ms/step - loss: 0.0494 - accuracy: 0.9824 - val_loss: 1.2310 - val_ac

In [17]:
train_y

<tf.Tensor: shape=(24432,), dtype=int64, numpy=array([1, 1, 1, ..., 0, 1, 1])>

In [ ]:


def plot_history(history):
  """
    Plotting training and validation learning curves.

    Args:
      history: model history with all the metric measures
  """
  fig, (ax1, ax2) = plt.subplots(2)

  fig.set_size_inches(18.5, 10.5)

  # Plot loss
  ax1.set_title('Loss')
  ax1.plot(history.history['loss'], label = 'train')
  ax1.plot(history.history['val_loss'], label = 'test')
  ax1.set_ylabel('Loss')

  # Determine upper bound of y-axis
  max_loss = max(history.history['loss'] + history.history['val_loss'])

  ax1.set_ylim([0, np.ceil(max_loss)])
  ax1.set_xlabel('Epoch')
  ax1.legend(['Train', 'Validation'])

  # Plot accuracy
  ax2.set_title('Accuracy')
  ax2.plot(history.history['accuracy'],  label = 'train')
  ax2.plot(history.history['val_accuracy'], label = 'test')
  ax2.set_ylabel('Accuracy')
  ax2.set_ylim([0, 1])
  ax2.set_xlabel('Epoch')
  ax2.legend(['Train', 'Validation'])

  plt.show()

plot_history(history)



## Model Evaluation

In [18]:
# Generate predictions on the dev set
predictions = model.predict(dev_x, batch_size=32)

# For a classification task, convert logits to predicted class labels.
# If using softmax activation, predictions will be probabilities.
pred_labels = np.argmax(predictions, axis=1)

# Print detailed classification metrics
print(classification_report(dev_y, pred_labels))

211/211 [==============================] - 34s 145ms/step
              precision    recall  f1-score   support

           0       0.76      0.67      0.71      3258
           1       0.72      0.80      0.76      3478

    accuracy                           0.74      6736
   macro avg       0.74      0.73      0.73      6736
weighted avg       0.74      0.74      0.73      6736



In [19]:
model.save('nli_bert_finetuned.h5', save_format='h5')

/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
